In [20]:
import numpy as np
import json
import os


tasks = []

n_qubits_range = list(range(1,11))
get_experiments_cnt = lambda n_qubits: 1000/(n_qubits**2)

def get_tensor_ranks(n_qubits: int, n:int=10):
    return list(set([int(x) for x in np.logspace(0, n_qubits+2, n, base=2)]))

algorithms = ['meg', 'dd_meg', 'cholesky']
data_types = ['pure', 'mixed']

task_id = 0
for n_qubits in n_qubits_range:
    experiments_cnt = get_experiments_cnt(n_qubits)
    for algorithm in algorithms:
        for data_type in data_types:
            task = {'id': task_id, 'algorithm': algorithm, 'n_qubits': n_qubits, 
                    'data_type': data_type, 'executions_cnt': int(experiments_cnt)}
            tasks.append(task)
            task_id += 1

for n_qubits in n_qubits_range:
    experiments_cnt = get_experiments_cnt(n_qubits)
    for tensor_rank in get_tensor_ranks(n_qubits):
        for data_type in data_types:
            task = {'id': task_id, 'algorithm': 'lptn', 'n_qubits': n_qubits, 'tensor_rank': tensor_rank, 
                    'data_type': data_type, 'executions_cnt': int(experiments_cnt)}
            tasks.append(task)
            task_id += 1
        
        
for file in os.listdir('../data/'):
    for algorithm in algorithms:
        task = {'id': task_id, 'algorithm': algorithm, 'executions_cnt': 1, 
                'file': f'../data/{file}', 'data_type': 'real'}
        tasks.append(task)
        task_id += 1

for file in os.listdir('../data/'):
    n_qubits = int(np.log2(int(file.split('_')[-2].replace('dim', ''))))
    for tensor_rank in get_tensor_ranks(n_qubits):
        task = {'id': task_id, 'algorithm': 'lptn', 'n_qubits': n_qubits, 'tensor_rank': tensor_rank, 
                'data_type': 'real', 'executions_cnt': 1}
        tasks.append(task)
        task_id += 1
        
with open('../experiments/tasks.json', 'w') as f:
    f.write(json.dumps(tasks, sort_keys=True, indent=4))

In [22]:
sum([t['executions_cnt'] for t in tasks])/600

54.92

In [45]:
import re
import subprocess

def get_gpu_memory(pid):
    output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode()
    for line in output.split('\n'):
        res = re.findall(f'.*\d*\s*{pid}\s*[CG]\s*.*python\s*(\d*)MiB.*', line)
        if res: return int(res[0]) # Mb

        
get_gpu_memory(13735)

545

In [47]:
import psutil

def get_ram_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024**2)  # in Mb

get_ram_memory_usage()

46.921875